In [1]:
import numpy as np
import pandas as pd
import os
import sys
import openpyxl
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.signal import savgol_filter
import re
import difflib
import pickle
import h5py
import itertools
import cv2
from pathlib import Path


In [2]:
from joblib import Parallel, delayed

In [3]:
import sys
sys.path.append("../project_behaviour/Code/")

from extract_angles import get_bend_angles, get_tan_angles, savgol_skeleton
from extract_curvatures import get_curv_savgol

In [ ]:
df_metadata_drugs = pd.read_pickle('/data/temp/athira/dlc_analysis/ref_metadataframe_drugs_Jan26.pickle')
df_metadata_drugs

In [4]:
df_metadata_dreadds = pd.read_pickle('/data/temp/athira/dlc_analysis/ref_metadataframe_dreadds_Feb06.pickle')
df_metadata_dreadds

,filepath,filename,drug,folder,drug_std,is_dechorionated,is_control,dissolved_in,with_CNO
8,/data/temp/athira/dlc_analysis/dlc_analysis_me...,20211112_085810_1_15m0s_CNO_VGLUThm4D_None_INV...,VGLUTKH.C3.324hm4DcherryCNO,dlc_analysis_meike_drugsWT_v4,VGLUTKH.C3.324hm4DcherryCNO,True,NaN,NaN,True
9,/data/temp/athira/dlc_analysis/dlc_analysis_me...,20211112_152313_1_15m0s_CNO_VGLUThm4D_None_INV...,VGLUTKH.C3.324hm4DcherryCNO,dlc_analysis_meike_drugsWT_v4,VGLUTKH.C3.324hm4DcherryCNO,True,NaN,NaN,True
10,/data/temp/athira/dlc_analysis/dlc_analysis_me...,20211112_131733_1_15m0s_CNO_VGLUThm4D_None_INV...,VGLUTKH.C3.324hm4DcherryCNO,dlc_analysis_meike_drugsWT_v4,VGLUTKH.C3.324hm4DcherryCNO,True,NaN,NaN,True
11,/data/temp/athira/dlc_analysis/dlc_analysis_me...,20211112_121207_1_15m0s_CNO_VGLUThm4D_None_INV...,VGLUTKH.C3.324hm4DcherryCNO,dlc_analysis_meike_drugsWT_v4,VGLUTKH.C3.324hm4DcherryCNO,True,NaN,NaN,True
12,/data/temp/athira/dlc_analysis/dlc_analysis_me...,20211112_152311_1_15m0s_CNO_VGLUThm4D_None_INV...,VGLUTKH.C3.324hm4DcherryCNO,dlc_analysis_meike_drugsWT_v4,VGLUTKH.C3.324hm4DcherryCNO,True,NaN,NaN,True
...,...,...,...,...,...,...,...,...,...
4919,/data/temp/athira/dlc_analysis/dlc_analysis_me...,20211005_162645_1_15m0s_CNO_THhM3D_None_INVERT...,THhM3DmcherryCNO,dlc_analysis_meike_drugsWT_v2,THhM3DmcherryCNO,True,NaN,NaN,True
4920,/data/temp/athira/dlc_analysis/dlc_analysis_me...,20211005_140538_1_15m0s_CNO_THhM3D_None_INVERT...,THhM3DmcherryCNO,dlc_analysis_meike_drugsWT_v2,THhM3DmcherryCNO,True,NaN,NaN,True
4921,/data/temp/athira/dlc_analysis/dlc_analysis_me...,20211005_135035_1_15m0s_CNO_THhM3D_None_INVERT...,THhM3DmcherryCNO,dlc_analysis_meike_drugsWT_v2,THhM3DmcherryCNO,True,NaN,NaN,True
4922,/data/temp/athira/dlc_analysis/dlc_analysis_me...,20211005_143719_1_15m0s_CNO_THhM3D_None_INVERT...,THhM3DmcherryCNO,dlc_analysis_meike_drugsWT_v2,THhM3DmcherryCNO,True,NaN,NaN,True


In [5]:
df_metadata_dreadds.drug_std.unique()

array(['VGLUTKH.C3.324hm4DcherryCNO', 'KH.C2.526VGAThm4Dcherry',
       'KH.C2.526VGAThm3Dcherry', 'VGLUTKH.C3.324hm4Dcherry',
       'KH.C2.526VGAThm4DcherryCNO', 'KH.C2.526VGAThm3DcherryCNO',
       'DDChM4Dmcherry', 'THhM3Dmcherry', 'VGLUTKH.C3.324hm3Dcherry',
       'THhM4Dmcherry', 'THhM3DmcherryCNO', 'VGLUTKH.C3.324hm3DcherryCNO',
       'THhM4DmcherryCNO', 'DDChM3Dmcherry', 'DDChM3DmcherryCNO',
       'DDChM4DmcherryCNO'], dtype=object)

In [ ]:
df_metadata_drugs.drug_std.unique()

In [ ]:
# len(df.)

In [6]:
eig_dict_july13 = np.load('../paleontology/eigen_dict_July13.npy', allow_pickle=True).item()

In [7]:
def segment_boundaries_lower(neck):
    
    tail_length = 48-(neck+2)+1
    tail_seg_len = int(np.round(tail_length / 5))
    
    segments_ind = {     
                'head' : 0,
                'neck' : (neck-1), 
                'tail_base': (neck+2),
                'tail_pre_mid': (neck+2)+ tail_seg_len,
                'tail_mid': (neck+2)+ 2*tail_seg_len,
                'tail_post_mid': (neck+2)+ 3*tail_seg_len,
                'tail_tip': (neck+2)+ 4*tail_seg_len
            }
    
#     max_skel_index = 48 # contour_width.shape[-1] - 1 # default : 48
#     segments_ind = { k: int(round(x*max_skel_index)) for k,x in segments_ind.items()}
    
    #Note : Here the indices are in 0 to 48 range
    return segments_ind



def segment_boundaries_upper(neck):
    
    tail_length = 48-(neck+2)+1
    tail_seg_len = int(np.round(tail_length / 5))
    
    segments_ind = {       
                'head' : (neck-2),
                'neck' : (neck+1), 
                'tail_base': ((neck+2)+ tail_seg_len)-1,
                'tail_pre_mid': ((neck+2)+ 2*tail_seg_len)-1,
                'tail_mid': ((neck+2)+ 3*tail_seg_len)-1,
                'tail_post_mid': ((neck+2)+ 4*tail_seg_len)-1,
                'tail_tip': 48
            }
    
#     max_skel_index = 48 
#     segments_ind = { k: int(round(x*max_skel_index)) for k,x in segments_ind.items()}
    
    #Note : Here the indices are in 0 to 48 range
    return segments_ind   


In [8]:
def calc_centroids_fw(neck_point):
    
    seg_ind_lower = segment_boundaries_lower(neck_point)
    seg_ind_upper = segment_boundaries_upper(neck_point)
    
    centroid_dict = {}
    
    for key in seg_ind_lower.keys():
        
        centroid_dict[key] = ((seg_ind_lower[key] + seg_ind_upper[key])//2)
    
    centroid_dict['neck'] = neck_point

    return centroid_dict

In [9]:
def create_features_dataframe(file_path, drug):
    
    try:
        df_skeleton = pd.read_hdf(file_path)
        print(len(df_skeleton.index),df_skeleton.isna().sum().sum())
        
        df_skeleton[df_skeleton < 0] = np.nan
        df_skeleton = df_skeleton[(df_skeleton.iloc[:, df_skeleton.columns.get_level_values(2) == 'likelihood'] > 0.6).all(axis=1)]
        df_skeleton = df_skeleton.dropna()
#         print((df_skeleton.max()),(df_skeleton.min()))
        print(len(df_skeleton.index),df_skeleton.isna().sum().sum())
    
        if len(df_skeleton.index) > 200:
    
            skel_array = df_skeleton.iloc[:,df_skeleton.columns.get_level_values(2)!='likelihood'].values.reshape(-1,49,2)

            curv_array = get_curv_savgol(skel_array)

            frame_indices = list(df_skeleton.index)
#             print(frame_indices)
            true_frame_indices = frame_indices#list(itertools.compress(range(len(frame_indices)), frame_indices)) 
            print(true_frame_indices[0:5])

            speed_array = np.diff(skel_array,prepend=skel_array[1,:,:].reshape(1,-1,2), axis = 0)
            delta_time = (np.diff(true_frame_indices, prepend=true_frame_indices[0]-1))
            print(delta_time)
            print(np.unique(delta_time))
            speed_array = np.divide(speed_array,delta_time[:,None,None])


            neck_pt = 12
            #lower bounds of segments
            lb_list = [segment_boundaries_lower(neck_pt)] * len(skel_array)
            #upper bounds of segments
            ub_list = [segment_boundaries_upper(neck_pt)] * len(skel_array)
            #centroids of segments
            centroid_list = [calc_centroids_fw(neck_pt)] * len(skel_array)

            # create a dataframe for an indexing operation down the line
            df_indices_lb = pd.DataFrame(lb_list)
            df_indices_ub = pd.DataFrame(ub_list)
            df_indices = pd.DataFrame(centroid_list)

            # List of segment names
            cols = list(centroid_list[0].keys())

            df_skel_file = pd.DataFrame()



            # segment wise : centroids
            for key in cols:

                # get the indices of the cenroid of the segment for all frames
                y = df_indices[key].values
                # get the indices of the lower boundary of the segment for all frames
                y_lb = df_indices_lb[key].values
                # get the indices of the upper boundary of the segment for all frames
                y_ub = df_indices_ub[key].values

                # get the valid frame indices (Elaborate: slack comment)
                ind_y = np.indices(y.shape)[0]
            #         print(skel_array.shape, curv_array.shape, skel_array.shape, curv_array.shape, 
            #               len(ind_y), len(y_lb), len(y_ub))  

                # Feature 1 : Curvature to the dataframe------------------
                df_skel_file[f"curv_{key}"] = curv_array[ind_y, y]

                # Feature 2 : speed and velocities------------------------
        #         df_skel_file[f"speed_x_{key}"] = speed_array[ind_y, y, 0]
        #         df_skel_file[f"speed_y_{key}"] = speed_array[ind_y, y, 1]
                df_skel_file[f"speed_{key}"] = np.linalg.norm(speed_array[ind_y, y, :], axis = 1)
                print(df_skel_file[f"speed_{key}"].max())

                # Feature 3 : Tangent angles--------------------------------
                # Calculate tan angles
                x_diff = skel_array[ind_y,y_ub,0]-skel_array[ind_y,y_lb,0]
                y_diff = skel_array[ind_y,y_ub,1]-skel_array[ind_y,y_lb,1]
                tan_array = np.arctan2(y_diff,x_diff)
                # To be done or not (attempt to make it unilateral - same side of the skeleton..)
                tan_array[tan_array < 0] += 2*np.pi
                df_skel_file[f"tan_{key}"] = tan_array

            # Feature 5 : Relative tangent angles using segments' tan angles
            for key in cols:
                if key != 'head':
                    df_skel_file[f"rel_tan_{key}"] = df_skel_file[f"tan_{key}"] - df_skel_file["tan_head"]

            # Feature 6 : EIGEN FEATURES
            curv_array_for_eig = curv_array[:,: -1]
            # Compute eigen coefficients
            eig_vec_matrix = np.array(eig_dict_july13['eig_vecs'][:,0:eig_dict_july13['n_modes']])   
            ects_test = np.dot(curv_array_for_eig, eig_vec_matrix)

            for i in range(ects_test.shape[1]):
                df_skel_file[f"ects_{i+1}"] = ects_test[:,i]


            # Feature 7 : calculate quirkiness
            dd = [cv2.minAreaRect(x) for x in skel_array.astype(np.float32)]
            dd = [(L,W) if L >W else (W,L) for _,(L,W),_ in dd]
            L, W = list(map(np.array, zip(*dd)))
            quirkiness = np.sqrt(1 - W**2 / L**2)
            df_skel_file['quirkness'] = quirkiness

            df_skel_file['frames'] = frame_indices#np.arange(len(skel_array))
            df_skel_file['filename'] = os.path.basename(file_path)
            df_skel_file['filedir'] = Path(file_path).parent.as_posix()
            df_skel_file['drug'] = drug
            return df_skel_file
        
        else:
            return None

    except FileNotFoundError:
        print(f'FileNotFoundError : {file_path}')
        return None

    except ValueError as er2:
        print(f'ValueError: {file_path}: {er2}')
        return None

    





## For drugs

In [ ]:
ind = 1200
create_features_dataframe(os.path.join(df_metadata_drugs['filepath'].iloc[ind],df_metadata_drugs['filename'].iloc[ind]),
                          df_metadata_drugs['drug_std'].iloc[ind]) 
                                                    

In [ ]:
df_features_list = Parallel(n_jobs=40, verbose = 5)(delayed(create_features_dataframe)(os.path.join(df_row['filepath'],df_row['filename']),df_row['drug_std']) 
                                                    for index, df_row in df_metadata_drugs.iterrows())
df_features_combined = pd.concat(df_features_list)

In [ ]:
# df_features_combined

In [ ]:
df_features_combined.to_pickle('/data/temp/athira/dlc_analysis/feats_combined_glutgaba_drugs_Jan31.pickle')

## For DREADDs

In [10]:
df_features_list_dreadds = Parallel(n_jobs=40, verbose = 5)(delayed(create_features_dataframe)(os.path.join(df_row['filepath'],df_row['filename']),df_row['drug_std']) 
                                                    for index, df_row in df_metadata_dreadds.iterrows())
df_features_combined_dreadds = pd.concat(df_features_list_dreadds)

[Parallel(n_jobs=40)]: Using backend LokyBackend with 40 concurrent workers.
[Parallel(n_jobs=40)]: Done  82 tasks      | elapsed:   15.0s
[Parallel(n_jobs=40)]: Done 208 tasks      | elapsed:   26.1s
[Parallel(n_jobs=40)]: Done 370 tasks      | elapsed:   41.8s
[Parallel(n_jobs=40)]: Done 568 tasks      | elapsed:   59.3s
[Parallel(n_jobs=40)]: Done 802 tasks      | elapsed:  1.4min
[Parallel(n_jobs=40)]: Done 1072 tasks      | elapsed:  1.9min
[Parallel(n_jobs=40)]: Done 1378 tasks      | elapsed:  2.4min
[Parallel(n_jobs=40)]: Done 1720 tasks      | elapsed:  3.0min
[Parallel(n_jobs=40)]: Done 1954 out of 1954 | elapsed:  3.4min finished


In [11]:
df_features_combined_dreadds.to_pickle('/data/temp/athira/dlc_analysis/feats_combined_glutgaba_dreadds_Feb06.pickle')

# Hampel filter

In [ ]:
from sktime.transformations.series.outlier_detection import HampelFilter

In [ ]:
def hampel_filter_skel(filepath):
    
    hampel_fn = f'/data/longterm/10/athira/HampelData/hampel_{os.path.basename(filepath)}'
    if not os.path.exists(hampel_fn):
        try:
            df_skel = pd.read_hdf(filepath)

        #     from sktime.transformations.series.outlier_detection import HampelFilter

            transformer = HampelFilter(window_length=10)
            df_skel_filt = df_skel.copy()

            for i in list(df_skel.columns.get_level_values(1)):

                test_x = df_skel.loc[:,('DLC_resnet50_Ciona_trackingMay25shuffle1_500000',i,'x')]
                test_y = df_skel.loc[:,('DLC_resnet50_Ciona_trackingMay25shuffle1_500000',i,'y')]
                df_skel_filt.loc[:,('DLC_resnet50_Ciona_trackingMay25shuffle1_500000',i,'x')] = transformer.fit_transform(test_x)
                df_skel_filt.loc[:,('DLC_resnet50_Ciona_trackingMay25shuffle1_500000',i,'y')] = transformer.fit_transform(test_y)

            df_skel_filt.to_pickle(f'/data/longterm/10/athira/HampelData/hampel_{os.path.basename(filepath)}')
        except ValueError as er:
            print(er, hampel_fn)
    return None


In [ ]:
len(df_metadata_drugs.index)

In [ ]:
df_hampel_list = Parallel(n_jobs=30, verbose = 5)(delayed(hampel_filter_skel)(os.path.join(df_row['filepath'],df_row['filename'])) 
                                                    for index, df_row in df_metadata_drugs.iterrows())

In [ ]:
test_path = os.path.join(df_metadata_drugs['filepath'][2801],df_metadata_drugs['filename'][2801])

In [ ]:
df_test_filt = hampel_filter_skel(test_path)

In [ ]:
df_test_filt

In [ ]:
test_path2 = f'/data/longterm/10/athira/HampelData/hampel_{os.path.basename(test_path)}'
test_path2

In [ ]:
df_test_filt.to_pickle(test_path2)

# Testing

In [ ]:
import random
for ind in random.sample(range(0, len(df_metadata_drugs.index)), 1):
    file_fullpath = os.path.join(list(df_metadata_drugs.filepath)[ind],list(df_metadata_drugs.filename)[ind])
    df_skeleton = pd.read_hdf(file_fullpath)
    
#     df_feat = create_features_dataframe(file_fullpath, list(df_metadata_drugs.drug)[ind])

In [ ]:
df_skeleton

In [ ]:
# df_skeleton[df_skeleton < 0] = np.nan
df_skeleton = df_skeleton[(df_skeleton.iloc[:, df_skeleton.columns.get_level_values(2) == 'likelihood'] > 0.6).all(axis=1)]
df_skeleton = df_skeleton.dropna()

In [ ]:
df_skeleton 

In [ ]:
skel_array_x = df_skeleton.iloc[:,df_skeleton.columns.get_level_values(2)=='x'].values
skel_array_y = df_skeleton.iloc[:,df_skeleton.columns.get_level_values(2)=='y'].values

In [ ]:
skel_array = df_skeleton.iloc[:,df_skeleton.columns.get_level_values(2)!='likelihood'].values.reshape(-1,49,2)           

In [ ]:
plt.scatter(skel_array_x[:,15],skel_array_y[:,15], cmap='jet', c=np.arange(0, len(skel_array_x)))

In [ ]:
# !pip install sktime

In [ ]:
from sktime.transformations.series.outlier_detection import HampelFilter
from sktime.datasets import load_airline

transformer = HampelFilter(window_length=10)
skel_array_hat = np.zeros_like(skel_array)
for i in range(skel_array.shape[1]):
    test_x = pd.Series(skel_array_x[:,i])
    test_y = pd.Series(skel_array_y[:,i])
    skel_array_x_hat = transformer.fit_transform(test_x)
    skel_array_y_hat = transformer.fit_transform(test_y)
    skel_array_hat[:,i,0]=skel_array_x_hat
    skel_array_hat[:,i,1]=skel_array_y_hat

plt.scatter(skel_array_hat[:,15,0],skel_array_hat[:,15,1], cmap='jet', c=np.arange(0, len(skel_array_x_hat)))

In [ ]:
curv_array =  get_curv_savgol(skel_array)
curv_array_hat =  get_curv_savgol(skel_array_hat)
fig, axes = plt.subplots(1,1, figsize = (15,4))

axes.plot(curv_array[:,15])
axes.plot(curv_array_hat[:,15])

In [ ]:
curv_array_hat =  get_curv_savgol(skel_array_hat)
plt.scatter(curv_array_hat[:,15])